# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f7d6b887950>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92693,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92694,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92695,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92696,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [45]:
# Set number of past days to use to make predictions
nb_lookback_days = 30
date_ls = []
geoid_ls = []
country_ls = []
newcase_ls = []
# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d]
        X_samples.append(X_sample)
        y_samples.append(y_sample)
        date_ls += [ list(gdf['Date'])[d] ]
        geoid_ls += [ list(gdf['GeoID'])[d] ]
        country_ls += [ list(gdf['CountryName'])[d]  ] 
        newcase_ls += [ list(gdf['NewCases'])[d]  ] 

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
import datetime
train_idx = [i for i in range(len(date_ls)) if date_ls[i] < datetime.date(2020,6,4) ]
test_idx = [i for i in range(len(date_ls)) if date_ls[i] >= datetime.date(2020,6,4) ]
train_idx = np.array(train_idx)
test_idx = np.array(test_idx)

In [15]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [16]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = X_samples[train_idx,:], X_samples[test_idx,:],y_samples[train_idx], y_samples[test_idx]
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape,)

(33250, 390) (15162, 390) (33250,) (15162,)


In [17]:
!pip install lightgbm

In [18]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
# model = Lasso(alpha=0.1,
#               precompute=True,
#               max_iter=10000,
#               positive=True,
#               selection='random')

# model = Lasso()
# # Fit model
# model.fit(X_train, y_train)

In [19]:
# change from Lasso to lightgbm

In [20]:
import random

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    
seed_everything(42) 

import lightgbm as lgb
NUM_ESTIMATORS = 10000
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'mae',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.05,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': NUM_ESTIMATORS,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# set negative values to zeros
y_train = np.maximum(y_train, 0)
train_data = lgb.Dataset(X_train,  label=y_train)
model = lgb.train(lgb_params,train_data,valid_sets = [train_data],verbose_eval = NUM_ESTIMATORS//20)

[500]	training's l1: 68.5453
[1000]	training's l1: 61.9273
[1500]	training's l1: 57.8149
[2000]	training's l1: 55.057
[2500]	training's l1: 52.7749
[3000]	training's l1: 50.6661
[3500]	training's l1: 49.0214
[4000]	training's l1: 47.4627
[4500]	training's l1: 46.1298
[5000]	training's l1: 44.8838
[5500]	training's l1: 43.7771
[6000]	training's l1: 42.7689
[6500]	training's l1: 41.8967
[7000]	training's l1: 40.9052
[7500]	training's l1: 40.14
[8000]	training's l1: 39.3991
[8500]	training's l1: 38.6785
[9000]	training's l1: 37.9648
[9500]	training's l1: 37.3506
[10000]	training's l1: 36.7355


In [40]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 36.73553494286659
Test MAE: 467.1284984355289


In [55]:
country_ls = np.array(country_ls)
geoid_ls = np.array(geoid_ls)
date_ls = np.array(date_ls)
newcase_ls = np.array(newcase_ls)

In [56]:
tf = pd.DataFrame(zip(country_ls[test_idx], geoid_ls[test_idx], date_ls[test_idx], newcase_ls[test_idx], test_preds ), columns=['CountryName','GeoID','Date','NewCases','NewCases_predict'])  
tf.to_csv('predict_from_June4.csv')

In [23]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
# for (col, coeff) in zip(x_col_names, list(model.coef_)):
#     if coeff != 0.:
#         print(col, coeff)
# print('Intercept', model.intercept_)

In [24]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [25]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [26]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 8.844880679844763
2020-08-02: 7.8900202953097605
2020-08-03: 5.420377405661566
2020-08-04: 7.68705182827371
2020-08-05: 5.209230450473871
2020-08-06: 4.604612886527648
2020-08-07: 5.210268896305745
2020-08-08: 6.962897858859115
2020-08-09: 8.37965282414986
2020-08-10: 5.349968414445907
2020-08-11: 5.602180923752734
2020-08-12: 4.861289372143064
2020-08-13: 5.871448828149551
2020-08-14: 4.799079577553682
2020-08-15: 6.51177911510402
2020-08-16: 7.467808427483098
2020-08-17: 7.2505007638562216
2020-08-18: 5.946674615075022
2020-08-19: 5.8976071385383015
2020-08-20: 7.8756102908953345
2020-08-21: 6.883286245944177
2020-08-22: 7.1888045825349645
2020-08-23: 7.514424463116519
2020-08-24: 8.243095197546836
2020-08-25: 6.615283032872335
2020-08-26: 7.077981513719826
2020-08-27: 8.762528763621917
2020-08-28: 8.155180872567572
2020-08-29: 7.243313595876185
2020-08-30: 6.681983438415381
2020-08-31: 7.4801209039738

Predicting for Afghanistan__nan
2020-08-01

2020-08-26: 243.9944048261193
2020-08-27: 285.0563638747212
2020-08-28: 178.61375022920348
2020-08-29: 178.66660849355898
2020-08-30: 181.64800721718598
2020-08-31: 125.0461846513799

Predicting for Burundi__nan
2020-08-01: 4.171325273814456
2020-08-02: 1.800047005824245
2020-08-03: 2.6067628984360947
2020-08-04: 2.645404488013806
2020-08-05: 3.150519364962355
2020-08-06: 2.9991279178454744
2020-08-07: 3.833331919444142
2020-08-08: 7.89369160285161
2020-08-09: 6.8278152264400465
2020-08-10: 10.445706940004902
2020-08-11: 13.536924513818256
2020-08-12: 26.476617469707865
2020-08-13: 24.61480781578088
2020-08-14: 34.0346015294566
2020-08-15: 27.893362397307396
2020-08-16: 32.83212513508555
2020-08-17: 37.896810393498214
2020-08-18: 45.41622946444843
2020-08-19: 52.845130860090805
2020-08-20: 74.67408569627555
2020-08-21: 74.51361703029671
2020-08-22: 87.12397200456167
2020-08-23: 105.40254113472969
2020-08-24: 109.44746901821058
2020-08-25: 150.57503618397826
2020-08-26: 145.595680267931

2020-08-31: 75.81835823320071

Predicting for Belarus__nan
2020-08-01: 914.1353877133106
2020-08-02: 978.2342623356648
2020-08-03: 983.9131202416917
2020-08-04: 1022.0028905335532
2020-08-05: 1112.0248627299945
2020-08-06: 989.0591445876214
2020-08-07: 726.399897236275
2020-08-08: 773.5188016354948
2020-08-09: 721.5863015101766
2020-08-10: 566.0171743963068
2020-08-11: 529.0342221703403
2020-08-12: 545.7678598951525
2020-08-13: 482.69667964165143
2020-08-14: 418.2881582359308
2020-08-15: 348.6334608773906
2020-08-16: 422.01671026474827
2020-08-17: 432.122736397593
2020-08-18: 409.41777851025506
2020-08-19: 357.3451222367441
2020-08-20: 294.20814567821117
2020-08-21: 159.6879460310165
2020-08-22: 226.82545794553403
2020-08-23: 176.79601594382908
2020-08-24: 244.58014684079336
2020-08-25: 204.34325591051564
2020-08-26: 197.1915913392436
2020-08-27: 203.50207049237005
2020-08-28: 156.55524931460192
2020-08-29: 153.86595052604915
2020-08-30: 175.51218402595956
2020-08-31: 117.3596428142389

2020-08-26: 159.96970028352715
2020-08-27: 166.03346315775147
2020-08-28: 183.587653409772
2020-08-29: 212.8781742030738
2020-08-30: 187.23448639770876
2020-08-31: 115.45614511517577

Predicting for Brazil__Ceara
2020-08-01: 528.7446589073388
2020-08-02: 715.3867729809873
2020-08-03: 583.8286410107027
2020-08-04: 278.252554570758
2020-08-05: 276.55429402493445
2020-08-06: 491.26409230516833
2020-08-07: 627.2113855471947
2020-08-08: 603.8337019897201
2020-08-09: 645.5971892273686
2020-08-10: 648.3525393103256
2020-08-11: 234.94735908774965
2020-08-12: 139.63815396485913
2020-08-13: 203.72182888363523
2020-08-14: 532.583856576611
2020-08-15: 831.9989813215736
2020-08-16: 697.7361677649589
2020-08-17: 373.88881722834066
2020-08-18: 201.77414025535165
2020-08-19: 139.01237755277936
2020-08-20: 137.04274019023907
2020-08-21: 205.01811055853548
2020-08-22: 347.20506699820794
2020-08-23: 375.2404142339516
2020-08-24: 436.36575175128917
2020-08-25: 229.16056749542793
2020-08-26: 204.8202861707

2020-08-23: 275.8315747450357
2020-08-24: 302.4357597161164
2020-08-25: 158.2975325893719
2020-08-26: 206.57219860437428
2020-08-27: 249.13678391831175
2020-08-28: 273.50128770776735
2020-08-29: 271.4340566100246
2020-08-30: 208.47058130430284
2020-08-31: 110.85037051280662

Predicting for Brazil__Paraiba
2020-08-01: 627.2058010509783
2020-08-02: 1122.8781593910899
2020-08-03: 556.6536759119671
2020-08-04: 315.7317084894214
2020-08-05: 345.50801361336465
2020-08-06: 348.73529178074136
2020-08-07: 386.97926884869037
2020-08-08: 444.91214687414737
2020-08-09: 564.9992662052765
2020-08-10: 324.1012562658611
2020-08-11: 157.05379122706594
2020-08-12: 135.38750856018405
2020-08-13: 309.12096850497227
2020-08-14: 451.3549850683341
2020-08-15: 570.8643834620399
2020-08-16: 662.4776994982294
2020-08-17: 236.81606716849333
2020-08-18: 107.57518683603183
2020-08-19: 128.55868931684697
2020-08-20: 155.74081355673246
2020-08-21: 192.65726196579874
2020-08-22: 308.2647337789973
2020-08-23: 319.7977

...

2020-08-23: 115.86997781626286
2020-08-24: 101.14214588647124
2020-08-25: 79.0379172857901
2020-08-26: 82.3507851764417
2020-08-27: 111.99682574624838
2020-08-28: 58.32363196728854
2020-08-29: 53.177901963801084
2020-08-30: 81.94741092762318
2020-08-31: 66.85092439350866

Predicting for United States__Mississippi
2020-08-01: 931.0531392517602
2020-08-02: 1318.5204480329926
2020-08-03: 1364.2451144406891
2020-08-04: 1205.0273211719166
2020-08-05: 1549.9033884049431
2020-08-06: 1151.9164216351696
2020-08-07: 875.6285940220685
2020-08-08: 896.1608046434766
2020-08-09: 1103.8838804820762
2020-08-10: 922.8887028277645
2020-08-11: 823.3066959698984
2020-08-12: 589.5354062707556
2020-08-13: 515.2522357594396
2020-08-14: 452.8032775969945
2020-08-15: 479.41754900084516
2020-08-16: 664.2896061336539
2020-08-17: 538.9631381769934
2020-08-18: 640.7195743500715
2020-08-19: 473.77720767798183
2020-08-20: 314.4321341113566
2020-08-21: 153.30239567780322
2020-08-22: 203.05763933472494
2020-08-23: 207

2020-08-10: 519.1027756800974
2020-08-11: 370.0411738193601
2020-08-12: 350.2877516011553
2020-08-13: 245.75968846478042
2020-08-14: 186.80392866576435
2020-08-15: 235.70642448437684
2020-08-16: 272.94839464671617
2020-08-17: 231.26420408954706
2020-08-18: 219.85248857738125
2020-08-19: 165.79035611639503
2020-08-20: 130.7226575359231
2020-08-21: 81.53062035204414
2020-08-22: 89.76264481819008
2020-08-23: 85.09245830287217
2020-08-24: 78.94873492001754
2020-08-25: 75.20601195126157
2020-08-26: 72.4717076411751
2020-08-27: 79.5436794666326
2020-08-28: 59.01731569325713
2020-08-29: 57.65382197959667
2020-08-30: 78.49987119183541
2020-08-31: 52.90978049758509

Predicting for United States__New York
2020-08-01: 3330.3964574149722
2020-08-02: 3250.1986982484946
2020-08-03: 3053.8975091547154
2020-08-04: 2765.2100619446564
2020-08-05: 3223.3360496441032
2020-08-06: 2891.1551510264285
2020-08-07: 2222.3340874683227
2020-08-08: 2081.542926616816
2020-08-09: 2478.020984079588
2020-08-10: 2768.9

2020-08-28: 49.57473178133214
2020-08-29: 39.913325046178386
2020-08-30: 51.818821264488655
2020-08-31: 36.867086625280166

Predicting for United States__Tennessee
2020-08-01: 1902.5366725835777
2020-08-02: 2875.119015976692
2020-08-03: 2679.2844708502585
2020-08-04: 1948.4740600533614
2020-08-05: 1928.3727731082906
2020-08-06: 2772.768364408488
2020-08-07: 1790.9968309602812
2020-08-08: 1609.2653096767124
2020-08-09: 1616.5554332724982
2020-08-10: 2121.584568118603
2020-08-11: 1681.3625796056588
2020-08-12: 1535.7912265323537
2020-08-13: 1164.5660060229347
2020-08-14: 1535.584027454371
2020-08-15: 1525.0485513333076
2020-08-16: 1180.1505659846046
2020-08-17: 1431.9288086541271
2020-08-18: 1510.4121514433077
2020-08-19: 1047.0473009713826
2020-08-20: 888.2303581976502
2020-08-21: 504.8732506961898
2020-08-22: 840.2099003592307
2020-08-23: 867.6962770656226
2020-08-24: 765.4798711421979
2020-08-25: 502.58250681314485
2020-08-26: 640.7901925664158
2020-08-27: 680.4885324352871
2020-08-28

2020-08-14: 255.2401756514724
2020-08-15: 232.90336797310877
2020-08-16: 396.60782440245066
2020-08-17: 475.10632842947206
2020-08-18: 359.15384512680555
2020-08-19: 217.8507482183641
2020-08-20: 223.36397867545273
2020-08-21: 126.33599846073619
2020-08-22: 145.4222661835675
2020-08-23: 143.28677712999487
2020-08-24: 167.9933365227593
2020-08-25: 153.11489263891016
2020-08-26: 146.91772133168283
2020-08-27: 179.61987961302452
2020-08-28: 131.28389624748257
2020-08-29: 137.19573923113234
2020-08-30: 185.9204460771976
2020-08-31: 101.63362440299733

Predicting for United States__Wyoming
2020-08-01: 380.3424696429232
2020-08-02: 195.71813585834326
2020-08-03: 167.32332674907485
2020-08-04: 228.23028013262382
2020-08-05: 191.8372883664425
2020-08-06: 230.85586836765225
2020-08-07: 217.1513319388659
2020-08-08: 233.16552149329144
2020-08-09: 218.72193827004915
2020-08-10: 163.70926894104744
2020-08-11: 164.18863997247476
2020-08-12: 134.72849010278293
2020-08-13: 107.6361830601352
2020-08-1

2020-08-02: 65.3709851941335
2020-08-03: 80.62596141031143
2020-08-04: 124.17147837332966
2020-08-05: 67.72438169753607
2020-08-06: 58.20837382156729
2020-08-07: 81.08937775184783
2020-08-08: 84.99448968569585
2020-08-09: 107.32659351595014
2020-08-10: 71.94411680864697
2020-08-11: 86.15472609613659
2020-08-12: 83.43772576840632
2020-08-13: 60.589483952888436
2020-08-14: 51.15590347681269
2020-08-15: 90.94077171668809
2020-08-16: 59.533061368667944
2020-08-17: 59.990937869067544
2020-08-18: 72.21882644355881
2020-08-19: 49.64250020585121
2020-08-20: 46.57879492395448
2020-08-21: 60.57504299731011
2020-08-22: 47.45041886220824
2020-08-23: 55.98934995590887
2020-08-24: 53.55548012290987
2020-08-25: 54.43143742289617
2020-08-26: 66.52454252129166
2020-08-27: 57.686807490558735
2020-08-28: 43.68499483556279
2020-08-29: 54.668089962579835
2020-08-30: 46.5794576534746
2020-08-31: 34.34445352281681
CPU times: user 16min 15s, sys: 1.26 s, total: 16min 17s
Wall time: 36.5 s


In [27]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,8.844881
214,Aruba,NaN,2020-08-02,7.890020
215,Aruba,NaN,2020-08-03,5.420377
216,Aruba,NaN,2020-08-04,7.687052
217,Aruba,NaN,2020-08-05,5.209230


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [28]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [29]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,8.844880679844763
Aruba,,2020-08-02,7.8900202953097605
Aruba,,2020-08-03,5.420377405661566
Aruba,,2020-08-04,7.68705182827371
Afghanistan,,2020-08-01,150.63176965201382
Afghanistan,,2020-08-02,93.04821396649523
Afghanistan,,2020-08-03,82.52804412470013
Afghanistan,,2020-08-04,77.02709660485289
Angola,,2020-08-01,106.90928989412532


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [30]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [31]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [32]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 267 ms, sys: 140 ms, total: 407 ms
Wall time: 7.49 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [33]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-22
End date: 2021-06-20


In [34]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [35]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-22 to 2021-06-20...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 7.65 s, sys: 1.16 s, total: 8.81 s
Wall time: 4min 13s
